In [5]:
!pip install PyPDF2
!pip install pandas

     -------------------------------------- 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [23]:
import PyPDF2
import pandas as pd
from io import StringIO

In [27]:
import fitz

In [9]:
pdf_file = open('PDF_sample.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

In [14]:
text = ''
for page in range(len(pdf_reader.pages)):
    text += pdf_reader.pages[0].extract_text()

In [24]:
df = pd.read_csv(StringIO(text), sep='\t', header=None)

In [25]:
df.to_excel('output.xlsx', index=False, header=False)

In [38]:
doc = fitz.open('PDF_sample.pdf')
page = doc[0]
words = page.get_text("words")

In [39]:
first_annots = []
rec = page.first_annot.rect

#Information of words in first object is stored in mywords

mywords = [w for w in words if fitz.Rect(w[:4]) in rec]
ann = page1.make_text(mywords)
first_annots.append(ann)

AttributeError: 'NoneType' object has no attribute 'rect'